<a href="https://colab.research.google.com/github/Alaaibrahim2/Alaa/blob/main/Language_Identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
df = pd.read_csv('/content/dataset.csv')
df

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna()

In [ ]:
df.isnull().sum()

In [ ]:
df['language'].value_counts()

In [ ]:
x = np.array(df['Text'])
y = np.array(df['language'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
count_vect = CountVectorizer()

In [ ]:
X = count_vect.fit_transform(x)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=3)

In [ ]:
model = MultinomialNB()
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
model=LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
model=RandomForestClassifier()
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D

In [ ]:
breakpoint = int(0.8 * len(df))
train_x = df['Text'][:breakpoint]
test_x = df['Text'][breakpoint:]

In [ ]:
ys = df['language']


In [ ]:
vocab_size = 40000
embedding_dim = 64
max_length = 150
trunc_type ='post'
oov_tok = '<OOV>'
num_unique_languages = len(set(ys))

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_x)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(train_x)
training_padded = pad_sequences(training_sequences, maxlen = max_length)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_x)
test_padded = pad_sequences(test_sequences, maxlen = max_length)

In [ ]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(ys)
word_index_y = label_tokenizer.word_index
label_sequences = label_tokenizer.texts_to_sequences(ys)
label_padded = pad_sequences(label_sequences)

In [ ]:
training_padded = np.array(training_padded)
test_padded = np.array(test_padded)
training_ys = np.squeeze(np.array(label_padded[:breakpoint]))
test_ys = np.squeeze(np.array(label_padded[breakpoint:]))
training_ys = training_ys - 1
test_ys = test_ys - 1
final_y_index = dict()
for language, value in word_index_y.items():
    final_y_index[language] = value - 1
word_index_y = final_y_index
del(final_y_index)
training_ys = to_categorical(training_ys, num_classes=num_unique_languages)
test_ys = to_categorical(test_ys, num_classes=num_unique_languages)

In [ ]:
import tensorflow.keras.layers as layers
model = Sequential([
    layers.Embedding(vocab_size, embedding_dim, input_length=max_length), # hyperparameters set above
    layers.GlobalAveragePooling1D(),
    layers.Dense(32, activation='relu'),
    layers.Dense(num_unique_languages, activation='sigmoid')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
num_epochs = 25
history = model.fit(training_padded, training_ys, epochs=num_epochs, validation_data=(test_padded, test_ys), verbose=2)

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
def predict(text, tokenizer, index_to_word_y, max_length, trunc_type):
    sequences = tokenizer.texts_to_sequences(text)
    padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
    predictions = model.predict(padded)
    for prediction in predictions:
        most_likely_lang = np.argmax(prediction)
        print(index_to_word_y[most_likely_lang])

In [ ]:
sentence = ['Pedro Sánchez Pérez-Castejón (Madrid, 29 de febrero de 1972) es un político español, actual presidente del Gobierno de España.', 
            'Hallo Hans. Wie geht es heute? Hast du Mittagessen gegessen?', # The model doesn't know German exists!
            'Emmanuel Macron, né le 21 décembre 1977 à Amiens, est un haut fonctionnaire, banquier d\'affaires et homme d\'État français. Il est président de la République française depuis le 14 mai 2017. ',
            'กรุงเทพมหานคร เป็นเมืองหลวงและนครที่มีประชากรมากที่สุดของประเทศไทย เป็นศูนย์กลางการปกครอง การศึกษา การคมนาคมขนส่ง การเงินการธนาคาร การพาณิชย์ การสื่อสาร และความเจริญของประเทศ ตั้งอยู่บนสามเหลี่ยมปากแม่น้ำเจ้าพระยา มีแม่น้ำเจ้าพระยาไหลผ่านและแบ่งเมืองออกเป็น 2 ฝั่ง คือ ฝั่งพระนครและฝั่งธนบุรี กรุงเทพมหานครมีพื้นที่ทั้งหมด 1,568.737 ตร.กม.',
            '你好!',
            'Gallia  est  omnis  dīvīsa  in  partēs  trēs,  quārum ūnam  incolunt  Belgae,  aliam Aquītānī,  tertiam  quī  ipsōrum  linguā  Celtae,nostrā  Gallī  appellantur.  '
           ]
index_to_word_y = dict([(value, key) for (key,value) in word_index_y.items()])
predict(sentence, tokenizer, index_to_word_y, max_length, trunc_type)

In [ ]:
max_words = 8192

In [ ]:
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim = max_words, output_dim = 256))
model_lstm.add(SpatialDropout1D(0.3))
model_lstm.add(LSTM(256, dropout = 0.3, recurrent_dropout = 0.3))
model_lstm.add(Dense(256, activation = 'relu'))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(5, activation = 'softmax'))
model_lstm.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

In [ ]:
num_epochs = 25
history = model.fit(training_padded, training_ys, epochs=num_epochs, validation_data=(test_padded, test_ys), verbose=2)

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
def predict(text, tokenizer, index_to_word_y, max_length, trunc_type):
    sequences = tokenizer.texts_to_sequences(text)
    padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
    predictions = model.predict(padded)
    for prediction in predictions:
        most_likely_lang = np.argmax(prediction)
        print(index_to_word_y[most_likely_lang])

In [ ]:
sentence = ['Pedro Sánchez Pérez-Castejón (Madrid, 29 de febrero de 1972) es un político español, actual presidente del Gobierno de España.', 
            'Hallo Hans. Wie geht es heute? Hast du Mittagessen gegessen?', # The model doesn't know German exists!
            'Emmanuel Macron, né le 21 décembre 1977 à Amiens, est un haut fonctionnaire, banquier d\'affaires et homme d\'État français. Il est président de la République française depuis le 14 mai 2017. ',
            'กรุงเทพมหานคร เป็นเมืองหลวงและนครที่มีประชากรมากที่สุดของประเทศไทย เป็นศูนย์กลางการปกครอง การศึกษา การคมนาคมขนส่ง การเงินการธนาคาร การพาณิชย์ การสื่อสาร และความเจริญของประเทศ ตั้งอยู่บนสามเหลี่ยมปากแม่น้ำเจ้าพระยา มีแม่น้ำเจ้าพระยาไหลผ่านและแบ่งเมืองออกเป็น 2 ฝั่ง คือ ฝั่งพระนครและฝั่งธนบุรี กรุงเทพมหานครมีพื้นที่ทั้งหมด 1,568.737 ตร.กม.',
            '你好!',
            'Gallia  est  omnis  dīvīsa  in  partēs  trēs,  quārum ūnam  incolunt  Belgae,  aliam Aquītānī,  tertiam  quī  ipsōrum  linguā  Celtae,nostrā  Gallī  appellantur.  '
           ]
index_to_word_y = dict([(value, key) for (key,value) in word_index_y.items()])
predict(sentence, tokenizer, index_to_word_y, max_length, trunc_type)